In [1]:
import pandas as pd
import numpy as np

## tokenizer and deleting punctuation

In [2]:
from corpy.morphodita import Tokenizer
tokenizer = Tokenizer("czech")
punc='[]{}()"„“!@#$%^=_&*-:/,.\''
def tokeny(clanek):
    cont=[]
    for word in tokenizer.tokenize(clanek):
        cont.append(word)
    nopunc = [char for char in cont if char not in punc]
    return nopunc

## lemmatizer

In [3]:
from corpy.morphodita import Tagger
tagger = Tagger("./czech-morfflex-pdt-161115.tagger")
def lemmatizer(slovo):
    tokens = list(tagger.tag(slovo))
    a=''.join(str(tokens))
    ind1=a.find('a=')
    ind2=a.find("'",ind1+3)
    return a[ind1+3:ind2]

## stopwords

In [4]:
filename = 'stopwords-cs.txt'
stopwords = np.loadtxt(filename, dtype=str,encoding="utf8")
def delete_stopwords(lematizovana_slova):
    return list(set(lematizovana_slova)-set(stopwords))

## load data

In [5]:
df=pd.ExcelFile('C:/Users/vacla/OneDrive/Dokumenty/Audiozprávy/article_archive_cleared.xls')

In [15]:
articles=df.parse('article_archive')
articles.head()

,title,link,summary,published,tags,category,credit,source
0,"Pokud chceme pandemii udržet pod kontrolou, je...",https://www.lidovky.cz/domov/pokud-chceme-pand...,Příznivý vývoj epidemie koronaviru v Česku lze...,2021-05-28 20:20:00,['Zprávy - Domov'],Zprávy z domova,ČTK,lidovky.cz
1,Hlavní manažer ANO pobíral neoprávněně plný za...,https://www.lidovky.cz/domov/hlavni-manazer-an...,Hlavní manažer hnutí ANO Jan Richter loni pobí...,2021-05-28 18:53:00,['Zprávy - Domov'],Zprávy z domova,MAFRA,lidovky.cz
2,"Vzpomeňme si, co udělaly bary loni v létě, mla...",https://www.lidovky.cz/domov/prilisne-rozvolne...,"Po zmírnění protiepidemických opatření, ke kte...",2021-05-28 18:03:00,['Zprávy - Domov'],Zprávy z domova,Petr Lundák MAFRA,lidovky.cz
3,Poslední „Berdychovec“ je za mřížemi. Policie ...,https://www.lidovky.cz/domov/policie-privezla-...,Čeští kriminalisté přivezli z USA Jaromíra Pro...,2021-05-28 18:03:00,['Zprávy - Domov'],Zprávy z domova,Policie ČR,lidovky.cz
4,Mezi Českem a sedmi zeměmi bude po první dávce...,https://www.lidovky.cz/domov/mezi-ceskem-a-sed...,"Ze Slovenska, Polska, Německa, Rakouska, Maďar...",2021-05-28 18:01:00,['Zprávy - Domov'],Zprávy z domova,ČTK,lidovky.cz


### looking for duplicated articles

SyntaxError: invalid syntax (Temp/ipykernel_9980/2082382247.py, line 1)

In [8]:
np.where(articles['source']!='lidovky.cz')

(array([ 300,  301,  302, ..., 5545, 5546, 5547], dtype=int64),)

In [34]:
articles.loc[29,'title']

'Prodej bytu Arenbergerovi prověřuje policie. Podle Hřiba by to měla řešit Praha 1, až pak magistrát'

In [30]:
articles.loc[60,'summary']

'Hokejisté Švédska i Kanady se výhrami na mistrovství světa v Rize udrželi ve hře o postup do čtvrtfinále. Každoroční kandidáti na medaile mají po pěti odehraných zápasech šest bodů a jsou momentálně mimo postupující čtveřici. Švédové jsou po výhře nad Velkou Británií 4:1 pátí ve skupině A, Kanada porazila Kazachstán 4:2 a je v béčku šestá.'

In [41]:
articles.loc[300,'summary']

'Adam Vojtěch je za necelý rok a půl pandemie koronaviru už pátým ministrem zdravotnictví. Je jeho návrat dobrým rozhodnutím? Poučí se ze svých chyb a dotáhne boj s pandemií covidu-19 do zdárného konce? V pořadu Pro a proti diskutovali šéfredaktor online Zdravotnického deníku Tomáš Cikrt a šéf Lékařského odborového klubu Martin Engel.'

In [32]:
cosine_similarity(train_x_vectors[60],train_x_vectors[342])

array([[0.30661118]])

### chose what features u want to use

In [7]:
#test
articles=articles[['title','summary']]

### prepare data, merge rows and create dataset

In [8]:
ddd=articles
#ddd['published']=np.datetime_as_string(ddd['published'])
for type in ddd.columns:
    ddd[type]=ddd[type].astype(str)

In [9]:
ddd.head()

,title,summary
0,"Pokud chceme pandemii udržet pod kontrolou, je...",Příznivý vývoj epidemie koronaviru v Česku lze...
1,Hlavní manažer ANO pobíral neoprávněně plný za...,Hlavní manažer hnutí ANO Jan Richter loni pobí...
2,"Vzpomeňme si, co udělaly bary loni v létě, mla...","Po zmírnění protiepidemických opatření, ke kte..."
3,Poslední „Berdychovec“ je za mřížemi. Policie ...,Čeští kriminalisté přivezli z USA Jaromíra Pro...
4,Mezi Českem a sedmi zeměmi bude po první dávce...,"Ze Slovenska, Polska, Německa, Rakouska, Maďar..."


### skip next cell if dont want to use time

In [10]:
for i in range(ddd.shape[0]):
    i1=ddd['published'][i][0:4]
    i2=ddd['published'][i][5:7]
    i3=ddd['published'][i][8:10]
    i4=ddd['published'][i][11:13]
    i5=ddd['published'][i][14:16]
    ddd['published'][i]=str(i1 + ' '+i2+' ' +i3+' ' +i4+' ' +i5 )
    #h=[i1,i2,i3,i4,i5]
    #for j in range(len(h)):
    #    ch=' '.join(h[i])
    #    ddd['published'].append(g)

KeyError: 'published'

In [11]:
data=[]
for i in range(ddd.shape[0]):
    g=' '.join(ddd.loc[i])
    data.append(g)

In [12]:
%%time
databaze=[]
d=[]
for clanek in data:
    d=tokeny(clanek)
    d=delete_stopwords(d) #change 1
    e=[]
    for slovo in d:
        e.append(lemmatizer(slovo))
    #e=delete_stopwords(e)
    databaze.append(e)
        

Wall time: 10.6 s


### to shape of CountVectorizer

In [13]:
dataset=[]
for index in range(len(databaze)):
    store=' '.join(databaze[index])
    dataset.append(store)


In [14]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
train_x_vectors=vectorizer.fit_transform(dataset)

In [18]:
from sklearn.metrics.pairwise import cosine_similarity,cosine_distances
res=cosine_similarity(train_x_vectors[1],train_x_vectors[0])

In [23]:
articles.loc[0:10,['title','summary','published','source']]


KeyError: "['published', 'source'] not in index"

In [151]:
articles.loc[446,['title','summary']]


title      Vakcína pro děti nad dvanáct let: Kdy mohou př...
summary    Zítra má Evropská léková agentura (EMA) doporu...
Name: 446, dtype: object

## duplicated articles
* 1 a 441
* 5 a 446
* 443,314,307,306,7,10
* 3 a 303
* 305 a 8
* 311 a 17 a 13
* 316 a 18
* 312 a 19
* 308 a 20
* sport
* 72 a 63
* 60 a 342
* 65 a 66 a 344 a 349 a 352
* 67 a 354
* 74 a 69
* 81 a 76 a 84
* 462 a 83
* 86 a 85

In [140]:
cosine_similarity(train_x_vectors[1],train_x_vectors[441])

array([[0.54352309]])

0-29 lid domov
60-89 lid sport
300-319 roz domov
340-359 roz sport
440-449 n cesko
460-469 n sport

In [184]:
trainx=np.concatenate((train_x_vectors[60:89].toarray(),train_x_vectors[340:359].toarray(),train_x_vectors[460:469].toarray()),axis=0)

In [185]:
%%time
cc=np.zeros([trainx.shape[0],trainx.shape[0]])
for i in range(trainx.shape[0]):
    for j in range(trainx.shape[0]):
        a=np.reshape(trainx[i],(1,25618))
        b=np.reshape(trainx[j],(1,25618))
        cc[i,j]=cosine_similarity(a,b)
    cc[i,i]=0
        

Wall time: 1.86 s


In [188]:
artic=np.concatenate((articles[60:89],articles[340:359],articles[460:469]),axis=0)

In [356]:
k=0.2
i=29

In [357]:
cc[np.where(cc[:,i]>k),i]

array([[0.30772873, 0.32623635, 0.36556308, 0.21011516, 0.33086205,
        0.30677372]])

In [358]:
artic[i]

array(['Hokejisté Dánska otočili zápas s Běloruskem a zatím jsou čtvrtí, Češi v tabulce klesli',
       'https://www.irozhlas.cz/sport/ms-hokej/dansko-belorusko-hokej-mistrovstvi-sveta-online-sledovani-ms-2021_2105281830_mim',
       'Hokejisté Dánska se při svém pátém vystoupení na mistrovství světa postavili proti Bělorusku a přestože soupeř ještě po dvou třetinách vedl, Seveřané dokázali v poslední části zápas otočit a nakonec zvítězili 5:2. Dánové se na turnaji posunuli na čtvrté místo neúplné tabulky skupiny A. Český tým klesl na šestou pozici, odehrál ale o utkání méně než Dánové i páté Švédsko.',
       Timestamp('2021-05-28 18:30:00'), '[]', 'Sport', nan,
       'irozhlas.cz'], dtype=object)

In [359]:
artic[np.where(cc[:,i]>k)]

array([['SOUHRN: Další trápení favoritů, Švédsko a Kanada však tentokrát vyhrály. Češi sraženi na šesté místo',
        'https://www.lidovky.cz/sport/hokej/svedi-porazili-tym-velke-britanie-4-1-kanada-po-vyhre-nad-kazachstanem-pokracuj.A210528_183530_ln-sport-hokej_lihem#utm_source=rss&utm_medium=feed&utm_campaign=ln_sport&utm_content=main',
        'Hokejisté Švédska i Kanady se výhrami na mistrovství světa v Rize udrželi ve hře o postup do čtvrtfinále. Každoroční kandidáti na medaile mají po pěti odehraných zápasech šest bodů a jsou momentálně mimo postupující čtveřici. Švédové jsou po výhře nad Velkou Británií 4:1 pátí ve skupině A, Kanada porazila Kazachstán 4:2 a je v béčku šestá.',
        Timestamp('2021-05-28 20:35:00'), "['Sport - Hokej']", 'Sport',
        'Reuters', 'lidovky.cz'],
       ['To byl obrat! Češi po dvou třetinách prohrávali 0:2, ale pak Švédům nasázeli čtyři góly',
        'https://www.lidovky.cz/sport/hokej/zive-svedsko-cesko-online-reportaz-hokej-mistrovstvi-s

In [217]:
np.where(cc[:,i]>k)

(array([12], dtype=int64),)

In [84]:
a=np.reshape(trainx[8],(1,25618))
b=np.reshape(trainx[34],(1,25618))
cosine_similarity(a,b)

array([[0.87148934]])

In [343]:
np.where(articles['title']=='Pešán po první výhře na MS: Volno vítám, chyb a nesystémové práce je hodně. Musíme zase najít chemii')

(array([85], dtype=int64),)

In [148]:
86 a 85

SyntaxError: invalid syntax (Temp/ipykernel_4356/3201865111.py, line 1)